In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\HAPPYUSER\\Documents\\Web_Development\\ML_AI_MLOPS\\Sports-image-classification'

In [4]:
import dagshub
import tensorflow as tf
dagshub.init(repo_owner='GauthamSU', repo_name='Sports-image-classification', mlflow=True)

Accessing as GauthamSU

Initialized MLflow to track repo "GauthamSU/Sports-image-classification"

Repository GauthamSU/Sports-image-classification initialized!

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    validation_folder: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from SportsImageClassifier.constants import *
from SportsImageClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            validation_folder="artifacts/data_ingestion/valid",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        validation_data = tf.keras.preprocessing.image_dataset_from_directory(
            self.config.validation_folder,
            image_size=self.config.params_image_size[:-1], 
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
            )
        self.scaled_val_data = validation_data.map(lambda x,y: (x/255, y))
        self.model = self.load_model(self.config.path_of_model)
        self.score = model.evaluate(self.scaled_val_data)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        dagshub.init(repo_owner='GauthamSU', repo_name='Sports-image-classification', mlflow=True)
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            mlflow.keras.log_model(self.model, "model", registered_model_name="MobileNetV2")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-09-03 00:35:16,106: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-03 00:35:16,112: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-03 00:35:16,114: INFO: common: created directory at: artifacts]
Found 500 files belonging to 100 classes.
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.0164 - loss: 4.6057
[2024-09-03 00:35:35,269: INFO: common: json file saved at: scores.json]


Initialized MLflow to track repo "GauthamSU/Sports-image-classification"

[2024-09-03 00:35:35,682: INFO: helpers: Initialized MLflow to track repo "GauthamSU/Sports-image-classification"]


Repository GauthamSU/Sports-image-classification initialized!

[2024-09-03 00:35:35,687: INFO: helpers: Repository GauthamSU/Sports-image-classification initialized!]


2024/09/03 00:35:36 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'MobileNetV2'.
2024/09/03 00:35:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MobileNetV2, version 1
Created version '1' of model 'MobileNetV2'.
2024/09/03 00:35:59 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/03 00:36:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run spiffy-shoat-888 at: https://dagshub.com/GauthamSU/Sports-image-classification.mlflow/#/experiments/0/runs/35b98660e2d64fab9ca766d47294d81b.
2024/09/03 00:36:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/GauthamSU/Sports-image-classification.mlflow/#/experiments/0.
